In [8]:
# from  pyspark.sql import SparkSession

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext

import pyspark.pandas as ps
import pandas as pd
import json
from datetime import datetime,date
import re
import numpy as np
import os

In [9]:
#spark=SparkSession.builder.master("local").appName("PySpark-Pandas").config("spark.jars", "postgresql-42.3.6.jar").getOrCreate()
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-19"
conf = SparkConf() \
    .setAppName("DemoSpark3.3") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/pyspark/*")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [10]:
today_x=datetime.now()
print(today_x)

2022-10-07 20:40:46.791289


In [11]:
sql_all="""

select  incident.id, incident.incident_no, product_type.productype_name,brand.brand_name,model.model_name
,incident.incident_severity_id,severity.severity_name,incident.incident_type_id,xtype.incident_type_name
,incident.incident_status_id,status.incident_status_name,incident.service_type_id,service.service_type_name
,incident.incident_customer_support

,inventory.serial_number,datacenter.datacenter_name
,incident.incident_subject,incident_description

,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as resolved_datetime



,TO_CHAR(inventory.install_date  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as install_date
,TO_CHAR(inventory.eos_date  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as eos_date

,TO_CHAR(inventory.customer_warranty_start  AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as customer_warranty_start
,TO_CHAR(inventory.customer_warranty_end AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as customer_warranty_end
, (select customer_name from app_customer 
  where id= (select id from app_customer where id=inventory.customer_support_id)
  ) as customer_support
  
,failure_type

from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id
inner join app_datacenter as datacenter on inventory.datacenter_id = datacenter.id
inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where company.id={}
and incident.incident_datetime>='{}'
and incident.incident_datetime<='{}'
and incident.incident_severity_id<>{}
and incident.incident_status_id <>{}

order by id

""".format(2,'2022-01-01','2022-12-31',4,3)

#'2022-01-01'   '2022-12-31'  4 3


In [12]:
def list_rows_to_spark_df(sql):
 sprk_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/SMartDB")\
    .option("query", sql) \
    .option("user", "postgres") \
    .option("password", "P@ssw0rd") \
    .option("driver", "org.postgresql.Driver") \
    .load()
 return sprk_df

incident_spdf=list_rows_to_spark_df(sql_all)
#incident_spdf.printSchema()

Py4JJavaError: An error occurred while calling o54.load.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:588)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvid

In [59]:
# convert spark_df to spark_ps
# main_cols=['incident_no','open_datetime','close_datetime','brand_name','model_name',\
#            'severity_name','incident_status_name']
df_all=incident_spdf.to_pandas_on_spark()  # convert datafram_spark to datafram_ps
#df_all=incident_spdf.toPandas() # convert datafram_spark to datafram_pd
df_all.info()


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 179 entries, 0 to 178
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         179 non-null    int32 
 1   incident_no                179 non-null    object
 2   productype_name            179 non-null    object
 3   brand_name                 179 non-null    object
 4   model_name                 179 non-null    object
 5   incident_severity_id       179 non-null    int32 
 6   severity_name              179 non-null    object
 7   incident_type_id           179 non-null    int32 
 8   incident_type_name         179 non-null    object
 9   incident_status_id         179 non-null    int32 
 10  incident_status_name       179 non-null    object
 11  service_type_id            179 non-null    int32 
 12  service_type_name          179 non-null    object
 13  incident_customer_support  179 non-null    object
 14  serial_

In [60]:
#df_all.head()

In [61]:
# #https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.to_datetime.html#pyspark.pandas.to_datetime
# df_all['open_datetime']=ps.to_datetime(df_all['open_datetime'].tolist(),format='%d %b %Y %H:%M').tolist()
# df_all['close_datetime']=ps.to_datetime(df_all['close_datetime'].tolist(), format='%d %b %Y %H:%M').tolist()

# df_all['response_datetime']=ps.to_datetime(df_all['response_datetime'].tolist(), format='%d %b %Y %H:%M').tolist()
# df_all['resolved_datetime']=ps.to_datetime(df_all['resolved_datetime'].tolist(), format='%d %b %Y %H:%M').tolist()
# # for calculating aging today-instaall date
# df_all['install_date']=ps.to_datetime(df_all['install_date'].tolist(), format='%d %b %Y %H:%M').tolist()
# df_all['eos_date']=ps.to_datetime(df_all['eos_date'].tolist(), format='%d %b %Y %H:%M').tolist()

# df_all.head(5)

In [26]:
#incident_type_id=14 name=Upgrade Software
def update_patch(x):
  if x==14:
    return  'Update patch'
  else:
    return  ''   

# df_all ['is_update_sw']=df_all['incident_type_id'].apply( lambda x : 'Update patch' if x==14 else '' )
# df_all.info()


In [62]:
df_all ['is_update_sw']=df_all['incident_type_id'].apply(update_patch)

In [64]:
s = ps.Series([20, 21, 12],
              index=['London', 'New York', 'Helsinki'])
s

London      20
New York    21
Helsinki    12
dtype: int64

In [66]:
def square(x) -> np.int64:
    return x ** 2


In [70]:
s.apply(square)

D:\ProgramData\Anaconda3\envs\myexp_base1\lib\site-packages\pyspark\sql\pandas\conversion.py:153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o3876.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at sun.reflect.GeneratedMethodAccessor69.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvo

Py4JJavaError: An error occurred while calling o3876.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at sun.reflect.GeneratedMethodAccessor69.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.Clie

In [69]:
s

London      20
New York    21
Helsinki    12
dtype: int64

In [79]:
df_all['month_year']= df_all['open_datetime'].apply(lambda x: x.strftime('%m-%Y')) 

D:\ProgramData\Anaconda3\envs\myexp_base1\lib\site-packages\pyspark\sql\pandas\conversion.py:153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o6744.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at sun.reflect.GeneratedMethodAccessor71.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvo

Py4JJavaError: An error occurred while calling o6744.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at sun.reflect.GeneratedMethodAccessor71.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.Clie

In [ ]:
_ILLEGAL_CHARACTERS_RE = re.compile(r"[\000-\010]|[\013-\014]|[\016-\037]")

# Apply function
df_all['incident_description2']=df_all['incident_description'].apply(lambda x: _ILLEGAL_CHARACTERS_RE.sub("", x))

In [196]:
df_all['status']= df_all['incident_status_id'].apply(lambda x:  'Closed' if x==4 else 'Opened')
# for pivot 

In [197]:
df_all['work_around_time_delta'] = df_all.apply(convert_datetime_to_timedelta, axis=1)

In [198]:
df_all['work_around_hour'] = df_all['work_around_time_delta'].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


In [199]:
#df_all['summary_work_around_str']=df_all['work_around_time_delta'].astype(object).where(df_all['work_around_time_delta'].notnull(),None)

df_all['summary_work_around_str']=df_all["work_around_time_delta"].apply( lambda x : str(x) )
df_all['summary_work_around_str']=df_all['summary_work_around_str'].apply( lambda x : x.replace('NaT','') )

In [200]:
def sla_in_out(item):
    # add service type= incident
    # critical and by 4 hour
    if item["incident_severity_id"]==1 :
       if item['work_around_hour']<=4:
        return "in"
       else:
        return "out"
        
    elif item["incident_severity_id"]==2:
       if item['work_around_hour']<=  168 :
        return "in"
       else:
        return "out"
    
    elif item["incident_severity_id"]==3:
       if item['work_around_hour']<=  (168 * 3) :
        return "in"
       else:
        return "out"
    else:
        return "cosmatic"
df_all['sla']=df_all.apply(sla_in_out,axis=1)   

In [201]:
def cal_againg(item):
    aging_year=0
    if  pd.isna(item["install_date"])==False  :
      aging_year= round( abs(today_x-item['install_date']).days/365,1)
      # print(f'{today_x} to {item["install_date"]}={aging_year}')
    else:
      aging_year= np.nan 
      # print (np.nan) 
    return aging_year

df_all['today']=today_x
df_all['aging_year']=df_all.apply( cal_againg,axis=1)

In [202]:
# get data from incidenet detail
sql_allDetail = """
select TO_CHAR(detail.task_start AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI')  as task_start
,TO_CHAR(detail.task_end AT TIME ZONE 'Asia/Bangkok','DD Mon YYYY HH24:MI') as task_end
,detail.workaround_resolution,team.service_team_name,engineer.employee_name as engineer_name
from app_incident_detail detail
     inner join  app_serviceteam team on detail.service_team_id=team.id
     inner  join  app_employee engineer on detail.employee_id=engineer.id
where detail.incident_master_id=%(incident_id_param)s  
ORDER BY detail.task_start DESC
"""



def all_detail_by_incident(row):
    
    id = row['id']
    df_detail = list_data(sql_allDetail,{"incident_id_param": id},get_postgres_conn())
    des = ''
    if df_detail.empty == False:
        for index, detail in df_detail.iterrows():
            task_start = detail['task_start']
            task_end = detail['task_end']
            x = f"{detail['service_team_name']} | {detail['engineer_name']} | {task_start} - {task_end} \n {detail['workaround_resolution']}\n\n"
            des = des + x

    des_fix = ''
    des_fix = _ILLEGAL_CHARACTERS_RE.sub("", des)

    return des_fix

df_all['detail'] = df_all.apply(all_detail_by_incident, axis=1)
#df_all['detail']=''


In [203]:
sql_caseIDDetail="""
select  app_incident_detail.incident_master_id as incident_id, app_incident_detail."reference_product_caseNo" as case_id from app_incident_detail
where app_incident_detail.incident_master_id  in %(incident_id_param)s 
"""

dfDetailCaseIDs=list_data(sql_caseIDDetail,{"incident_id_param": tuple(df_all['id'].tolist())},get_postgres_conn())
dfDetailCaseIDs.dropna(axis=0, how='any',inplace=True)
dfDetailCaseIDs.drop_duplicates(inplace=True)
dfDetailCaseIDs=dfDetailCaseIDs.groupby(['incident_id'], as_index=False).agg({'case_id' : ','.join  })

#dfDetailCaseIDs

df_all=df_all.merge(dfDetailCaseIDs,left_on="id",right_on="incident_id",how='left')

In [204]:
# 982=for 4 items and 2 team but 3 eng ,987 =no enginerr ,883= for 2 items and 1 team  and 1 eng

sql_count_eng="""
select app_incident_detail.incident_master_id as incident_id, count(distinct employee_id) as "no.eng"  from app_incident_detail
where app_incident_detail.incident_master_id in %(incident_id_param)s 
group by  app_incident_detail.incident_master_id
"""

dfNoEng=list_data(sql_count_eng,{"incident_id_param": tuple(df_all['id'].tolist())},get_postgres_conn())
# dfNoEng
df_all=df_all.merge(dfNoEng,left_on="id",right_on="incident_id",how='left')

df_all["no.eng"] = df_all["no.eng"].fillna(0)

In [205]:
df_all['issue_datetime']=df_all['open_datetime']
df_all=df_all.sort_values(by=['issue_datetime'],ascending=True)

In [206]:
for d in date_cols:
 df_all[d]=df_all[d].dt.strftime('%d-%b-%y')

In [207]:
for d in datetime_cols:
 df_all[d]=df_all[d].dt.strftime('%d-%b-%y %H:%M')

# splite all issues dataframe to others 

In [208]:
print("Create incident issues dataframe")
#service_type=incident
dfIncident=df_all.query("service_type_id==1")

dfIncident=dfIncident.reset_index(drop=True)
dfIncident = dfIncident.reset_index(level=0)
dfIncident.rename(columns={"index": "running_number"},inplace=True)
dfIncident['running_number']=dfIncident['running_number']+1


Create incident issues dataframe


In [209]:
print("Create Out-SLA incident issues dataframe")
outSLA_str='out'
dfOutSLA=dfIncident.query("sla==@outSLA_str")

Create Out-SLA incident issues dataframe


In [210]:
dfOutSLA['problem']=dfOutSLA.apply( lambda x :  f"{x.incident_subject}\n\n{x.incident_description}",axis=1 )
dfOutSLA['cause']=''
dfOutSLA['effect']=''
dfOutSLA['solution']=''
dfOutSLA['preventive_guideline']=''
dfOutSLA['vender']='Yip In Tsoi'

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_15112\242617438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutSLA['problem']=dfOutSLA.apply( lambda x :  f"{x.incident_subject}\n\n{x.incident_description}",axis=1 )
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_15112\242617438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOutSLA['cause']=''
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_15112\242617438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [211]:
#service_type=incident
print("Create request service to maintainance dataframe")
dfService=df_all.query("service_type_id==2")
dfService=dfService.reset_index(drop=True)
dfService = dfService.reset_index(level=0)
dfService.rename(columns={"index": "running_number"},inplace=True)
dfService['running_number']=dfService['running_number']+1

#incident type 15=Report
dfService['service_type_name']= dfService.apply( lambda x: 'Report' if x.incident_type_id==15 else x.service_type_name ,axis=1)

Create request service to maintainance dataframe


# Export Excel Report as Customer Format & Name

# All Issue

In [212]:
dfAllIssue=df_all[dfAll_Issue_Cols['name'].tolist()]
dfAllIssue=dfAllIssue.rename(columns=dictAll_Issue)
dfAllIssue=dfAllIssue[dfAll_Issue_Cols['display_name'].tolist()]

In [213]:
dfIncident=dfIncident[dfIncident_Issue_Cols['name'].tolist()]
dfIncident=dfIncident.rename(columns=dictIncident)
dfIncident=dfIncident[dfIncident_Issue_Cols['display_name'].tolist()]

In [214]:
dfOutSLA=dfOutSLA[dfOutSLA_Cols['name'].tolist()]
dfOutSLA=dfOutSLA.rename(columns=dictOutSLA)
dfOutSLA=dfOutSLA[dfOutSLA_Cols['display_name'].tolist()]

In [215]:
dfService=dfService[dfService_Issue_Cols['name'].tolist()]
dfService=dfService.rename(columns=dictService)
dfService=dfService[dfService_Issue_Cols['display_name'].tolist()]

In [218]:
# dev eviroemnt
if is_production==False:
    print("Developement result")
 
    writer=pd.ExcelWriter("AIS-Table-Report.xlsx",engine='xlsxwriter') 

    dfAllIssue.to_excel(writer, sheet_name="AllIssues",index=False)
    dfIncident.to_excel(writer, sheet_name="IncidentIssues",index=False)
    dfOutSLA.to_excel(writer, sheet_name="OutSLA",index=False)
    dfService.to_excel(writer, sheet_name="PreventiveServiceRequest",index=False)
    writer.save()
   
    df_all.to_excel('df_all.xlsx',index=False)
    #return None,None
# production
else:
    print("Production result")
    dfTableReportDict={  \
        "AllIssues": dfAllIssue, \
        "IncidentIssues": dfIncident, \
        "OutSLA": dfOutSLA, \
        "PreventiveServiceRequest": dfService    

    }
    #return df_all,dfTableReportDict

Production result


In [219]:
for sheet_name,data  in dfTableReportDict.items():
    print(sheet_name)

AllIssues
IncidentIssues
OutSLA
PreventiveServiceRequest
